# Wine Quality -TF Model

In [1]:
# Import the necessary libraries
import os

import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from sklearn import metrics, preprocessing
from sklearn.model_selection import train_test_split

## Data Preparation
### Load the data
Read the cleanded data into a Pandas DataFrame.

In [2]:
df = pd.read_csv('data/winequality-red-cleaned.csv', sep=',')

This dataset consists of the following columns:

In [3]:
print(df.columns.values)

['fixed acidity' 'volatile acidity' 'citric acid' 'residual sugar'
 'chlorides' 'free sulfur dioxide' 'total sulfur dioxide' 'density' 'pH'
 'sulphates' 'alcohol' 'quality' 'category']


### Labels and Features

In [4]:
features_list = df.columns.values[:-2]
labels_column = df.columns.values[-1]
print('The features are: {}'.format(features_list))
print('The label column is: {}'.format(labels_column))

The features are: ['fixed acidity' 'volatile acidity' 'citric acid' 'residual sugar'
 'chlorides' 'free sulfur dioxide' 'total sulfur dioxide' 'density' 'pH'
 'sulphates' 'alcohol']
The label column is: category


Split the dataset into features and labels. Labels will be convert from string values to an integer.

In [5]:
y = [0 if item == 'Good' else 1 for item in df['category']]
print(y[0:5])

[1, 1, 1, 0, 1]


In [6]:
X = df.drop(['quality', 'category'], axis=1).values

#### One-Hot Vectors
In this implementation of TensorFlow the labels need to be expressed as one-hot vectors. A one-hot vector has length equal to the number of catetories and values of 0 and 1. For examples, category value **Good** == 0 => **`[0, 1]`**, while **Bad** == 1 => **`[1, 0]`**.

Here I define a function to convert the numeric labels into one-hot vectors.

In [7]:
def dense_to_one_hot(labels_dense, num_classes=2):
    labels_one_hot = []
    for label in labels_dense:
        indices = [1]*num_classes
        indices[label] = 0
        labels_one_hot.append(indices)
    
    return labels_one_hot

In [8]:
y_one_hot = dense_to_one_hot(y, num_classes=2)
print(y_one_hot[0:5])

[[1, 0], [1, 0], [1, 0], [0, 1], [1, 0]]


### Train/Test Split

Split the data into a training set and a test set.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

## Model Preparation

### Hyperparameters
These are parameters that can be changed to modify how the model is fit. 

In [10]:
learning_rate = 0.001
batch_size = X_train.shape[0] // 10
num_features = X_train.shape[1]
num_classes = 2
epochs = 1000
epochs_to_print = epochs // 10
hidden_layer_units = 20

### Placeholders
TensorFlow stores fixed quantities, like the features and labels, in a **`placeholder()`**.

In [11]:
X_placeholder = tf.placeholder(tf.float32, [None, num_features], name='X')
y_placeholder = tf.placeholder(tf.float32, [None, num_classes], name='y')

### Batches
The model is trained by taking random sambles from the training set. 

In [12]:
def make_batch(X, y, batch_size):
    
    y_size = len(y)
    index_sample = np.random.choice(y_size, batch_size, replace=False)
    y_array = np.array(y)
    
    X_batch = X[index_sample]
    y_batch = y_array[index_sample]
    
    return X_batch, y_batch

### Softmax Layer
Use softmax regression to model the data. Set up the model weights and biases.

In [13]:
def softmax_layer(X_tensor, num_units):
    num_inputs = X_tensor.get_shape()[1].value
    W = tf.Variable(tf.zeros([num_inputs, num_units]), name='W')
    b = tf.Variable(tf.zeros([num_units]), name='b')
    y = tf.nn.softmax(tf.matmul(X_tensor, W) + b)
    return y

### ReLu Layer
Model complexity can be increased by adding additional layers. Here I define a function to create a ReLu layer, which can be added to the graph.

In [14]:
def relu_layer(X_tensor, num_units):
    num_inputs = X_tensor.get_shape()[1].value
    W = tf.Variable(tf.zeros([num_features, num_units]), name='W')
    b = tf.Variable(tf.zeros([num_units]), name='b')
    y = tf.nn.relu(tf.matmul(X_tensor, W) + b, name='relu')
    return y

### Cost Function

Minimize the error using cross entropy.

In [15]:
def define_cost_function(y, y_tensor):
    cost = -tf.reduce_sum(y_tensor * tf.log(y), name='cross_entropy')
    return cost

### Training Step
Define the optimizer. I use gradient descent.

In [16]:
def train(cost, learning_rate):
    training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    return training_step

### Accuracy
This function will be used to compute model accuracy during training and at the end.

In [17]:
def compute_accuracy(y, y_tensor):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_tensor, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"), name='accuracy')
    return accuracy

## TensorFlow Graph
Now that we have defined functions to make the TensorFlow graph, here is where we actually create it.

In [18]:
def single_layer():
    # Create softmax layer
    with tf.name_scope("softmax") as scope:
        y_softmax = softmax_layer(X_placeholder, num_classes)

    # Define cost function
    with tf.name_scope("cost_function") as scope:
        global cost
        cost = define_cost_function(y_softmax, y_placeholder)
        tf.summary.scalar("cost", cost)

    # Define training step
    with tf.name_scope("training") as scope:
        global training_step
        training_step = train(cost, learning_rate)

    # Calculate model accuracy
    with tf.name_scope("accuracy") as scope:
        global accuracy
        accuracy = compute_accuracy(y_softmax, y_placeholder)
        tf.summary.scalar("accuracy", accuracy)

In [19]:
def two_layers():
    # Create hidden, relu layer
    with tf.name_scope("hidden_layer") as scope:
        y_relu = relu_layer(X_placeholder, hidden_layer_units)
    
    # Create softmax layer
    with tf.name_scope("softmax") as scope:
        y_softmax = softmax_layer(y_relu, num_classes)

    # Define cost function
    with tf.name_scope("cost_function") as scope:
        global cost
        cost = define_cost_function(y_softmax, y_placeholder)
        tf.summary.scalar("cost", cost)

    # Define training step
    with tf.name_scope("training") as scope:
        global training_step
        training_step = train(cost, learning_rate)

    # Calculate model accuracy
    with tf.name_scope("accuracy") as scope:
        global accuracy
        accuracy = compute_accuracy(y_softmax, y_placeholder)
        tf.summary.scalar("accuracy", accuracy)

In [20]:
single_layer()
# two_layers()

## Model Training
A TensorFlow session is started and the model is trained using the graph that we setup above.

In [21]:
# Merge summaries for TensorBoard
merged_summaries = tf.summary.merge_all()

with tf.Session() as sess:

    log_directory = 'tmp/logs'
    summary_writer = tf.summary.FileWriter(log_directory, sess.graph)
    
    tf.global_variables_initializer().run()
    
    # average_cost = 0
    cost_sum = 0
    for i in range(epochs):
        
        X_batch, y_batch = make_batch(X_train, y_train, batch_size)
        feed_dict = {X_placeholder: X_batch, y_placeholder: y_batch}
        _, current_cost = sess.run([training_step, cost], feed_dict)
        cost_sum += current_cost
        
        # Print average cost periodically
        if i % epochs_to_print == 99:
            average_cost = cost_sum / epochs_to_print
            print("Epoch: {:4d}, average cost = {:0.3f}".format(i+1, average_cost))
            cost_sum = 0
    
    print('Finished model fitting.')
 
    # Calculate final accuracy
    X_batch, y_batch = make_batch(X_test, y_test, batch_size)
    feed_dict = {X_placeholder: X_test, y_placeholder: y_test}
    print("Final accuracy = {:0.3f}".format(sess.run(accuracy, feed_dict)))

Epoch:  100, average cost = 68.633
Epoch:  200, average cost = 66.298
Epoch:  300, average cost = 65.915
Epoch:  400, average cost = 64.442
Epoch:  500, average cost = 65.069
Epoch:  600, average cost = 65.406
Epoch:  700, average cost = 66.335
Epoch:  800, average cost = 65.926
Epoch:  900, average cost = 65.367
Epoch: 1000, average cost = 65.455
Finished model fitting.
Final accuracy = 0.750
